# Serotonin 3D GNN Project


This project builds upon research done by Łapińska et al. (2024): https://doi.org/10.3390/pharmaceutics16030349

Data used: https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/releases/chembl_35/

Move the unpacked chembl_35_sqlite.tar.gz file into the data/ dir.

The research linked above presents two Quantitative Structure-Activity Relationship (QSAR) models to predict serotonergic binding affinity and selectivity, respectively, using Mordred molecular 2D descriptors. Specifically, one model classifies compounds binarily as "active" or "inactive", with a cutoff of pKi = 7. Another model does multiclass classification to predict the serotonergic selectivity of compounds previously classified as "active".

I am following a similar approach, but using 3D molecular graph representations instead of 2D molecular descriptors as input modality and using only the ChEMBL database, not ZINC.


## Google Colab Setup


### Configuration


In [1]:
from pathlib import Path

try:
    from google.colab import drive

    drive.mount("/content/drive")
    IN_COLAB = True
except:
    IN_COLAB = False

print(f"{'Running in Colab' if IN_COLAB else 'Running locally'}")

PATH_NOTEBOOK = (
    Path("/content/drive/MyDrive/Colab Notebooks/serotonin-3d-gnn.ipynb")
    if IN_COLAB
    else Path(
        "/Users/paul/Library/CloudStorage/GoogleDrive-unoutsch@gmail.com/My Drive/Colab Notebooks/serotonin-3d-gnn.ipynb"
    )
)
PATH_REPO = (
    Path("/content/drive/MyDrive/Repositories/serotonin-3d-gnn")
    if IN_COLAB
    else Path.cwd()
)
PATH_DATA = PATH_REPO / "data"

Running locally


### Syncing Google Drive with Google Colab Content


### Installing Requirements


In [3]:
%pip install -r "$PATH_REPO/requirements.txt"

Note: you may need to restart the kernel to use updated packages.


## Imports


In [3]:
import os
import sys
import time

import pandas as pd
import pickle
from rdkit import Chem
from rdkit.Chem import AllChem, rdchem, HybridizationType
import shutil
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

## Utils


### Syncing this file between Colab and local Git repo


Make sure the paths exist.


In [107]:
if IN_COLAB:
    shutil.copyfile(PATH_NOTEBOOK, PATH_REPO / "serotonin-3d-gnn.ipynb")
    print("Copied notebook to repo.")
else:
    shutil.copyfile(PATH_REPO / "serotonin-3d-gnn.ipynb", PATH_NOTEBOOK)
    print("Copied notebook to Google Drive.")

Copied notebook to Google Drive.


## Data


### Note on Data Aquisition from chembl_35.db

In order to collect the desired data from the ChEMBL SQL database and transform it into a .csv file, I undertook the steps detailed in `data/README.md`.


### Loading the Data


If the pickled torch_data_list already exists, load it.


In [14]:
pickle_file_path = PATH_DATA / "torch_data_list.pkl"

if os.path.exists(pickle_file_path):
    torch_data_list = pickle.load(open(pickle_file_path, "rb"))
    print("Loaded torch_data_list from pickle file")
else:
    print("Creating torch_data_list from scratch")

df = pd.read_csv(PATH_DATA / "serotonin_binding_summary.csv")

df.describe()

Loaded torch_data_list from pickle file


,molecule_id,Serotonin (5-HT) receptor,Serotonin 1 (5-HT1) receptor,Serotonin 1 receptors; 5-HT1B & 5-HT1D,Serotonin 1a (5-HT1a) receptor,Serotonin 1b (5-HT1b) receptor,Serotonin 1d (5-HT1d) receptor,Serotonin 1e (5-HT1e) receptor,Serotonin 1f (5-HT1f) receptor,Serotonin 2 (5-HT2) receptor,...,Serotonin 2b (5-HT2b) receptor,Serotonin 2c (5-HT2c) receptor,Serotonin 3 (5-HT3) receptor,Serotonin 3a (5-HT3a) receptor,Serotonin 3b (5-HT3b) receptor,Serotonin 4 (5-HT4) receptor,Serotonin 5a (5-HT5a) receptor,Serotonin 5b (5-HT5b) receptor,Serotonin 6 (5-HT6) receptor,Serotonin 7 (5-HT7) receptor
count,2.345600e+04,90.000000,252.000000,1.0,9462.000000,1492.000000,1472.000000,91.000000,127.000000,1469.000000,...,2337.000000,4343.000000,939.000000,1040.000000,8.000000,1009.000000,422.000000,1.00,4221.000000,3100.000000
mean,1.003325e+06,6.081759,6.683902,6.2,7.258523,6.952528,7.554968,5.791172,7.458423,7.053201,...,6.603829,6.810210,7.625768,7.047520,7.203500,7.645809,6.573801,7.17,7.311171,6.977487
std,8.986583e+05,0.926906,1.104283,NaN,1.152004,1.226482,1.365880,0.652239,0.859240,1.159567,...,0.981462,1.032874,1.225942,1.535413,1.735342,1.179482,1.089819,NaN,1.143388,1.016128
min,9.700000e+01,4.390000,4.100000,6.2,4.000000,4.000000,4.000000,4.800000,5.140000,4.030000,...,4.190000,4.000000,4.010000,4.000000,5.460000,5.000000,4.070000,7.17,4.120000,4.000000
25%,2.291570e+05,5.385000,5.800000,6.2,6.480000,6.050000,6.470000,5.360000,7.000000,6.240000,...,5.900000,6.050000,6.800000,5.700000,5.490000,6.810000,5.800000,7.17,6.470000,6.285000
50%,5.757615e+05,5.925000,6.600000,6.2,7.280000,6.850000,7.640000,5.730000,7.720000,6.920000,...,6.523333,6.740000,7.700000,7.185000,7.015000,7.640000,6.390000,7.17,7.360000,6.990000
75%,1.965967e+06,6.657500,7.585000,6.2,8.060000,7.850000,8.700000,6.175000,8.020000,8.000000,...,7.210000,7.512500,8.580000,8.410000,8.393500,8.400000,7.068750,7.17,8.110000,7.700000
max,2.881244e+06,9.800000,9.300000,6.2,11.000000,10.000000,10.700000,8.200000,8.800000,10.300000,...,10.100000,10.700000,10.420000,10.400000,9.604000,10.800000,9.170000,7.17,10.400000,10.000000


In [15]:
df_targets = df.drop(columns=["molecule_id", "canonical_smiles"])
print(f"Number of targets: {len(df_targets.columns)}")

# compute non-nan counts per column and create a boolean mask to filter targets to contain at least n_threshold non-nan values
n_threshold = 1000
non_nan_counts = torch.tensor(df_targets.notna().sum().values, dtype=torch.long)
mask = non_nan_counts >= n_threshold
valid_column_indices = torch.nonzero(mask, as_tuple=True)[0]

print("Boolean mask:", mask)
print(f"Included targets ({len(valid_column_indices)}):", valid_column_indices)

Number of targets: 21
Boolean mask: tensor([False, False, False,  True,  True,  True, False, False,  True, False,
         True,  True,  True, False,  True, False,  True, False, False,  True,
         True])
Included targets (11): tensor([ 3,  4,  5,  8, 10, 11, 12, 14, 16, 19, 20])


In [ ]:
periodic_table = rdchem.GetPeriodicTable()

ATOM_PROPERTIES = {
    atomic_num: [
        periodic_table.GetAtomicWeight(atomic_num),
        periodic_table.GetRvdw(atomic_num),
        periodic_table.GetDefaultValence(atomic_num),
    ]
    for atomic_num in range(1, 119)  # all elements in periodic table
}

BOND_TYPES = [
    Chem.rdchem.BondType.SINGLE,
    Chem.rdchem.BondType.AROMATIC,
    Chem.rdchem.BondType.DOUBLE,
    Chem.rdchem.BondType.TRIPLE,
]


def create_torch_data(smiles: str, targets: torch.Tensor) -> Data:
    # getting RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    Chem.Molf

    if mol is None:
        return None

    # add explicit hydrogen atoms to the molecule (are not included in the SMILES string) so that its 3D structure is complete
    mol = Chem.AddHs(mol)

    # EmbedMolecule positions atoms of mol in 3D space stochastically; if it fails (returning -1) return None
    if AllChem.EmbedMolecule(mol, randomSeed=42) == -1:
        return None

    # optimize the 3D structure using Universal Force Field (UFF) to lower mol's energy
    AllChem.UFFOptimizeMolecule(mol)

    # conformer contains 3D coordinates for mol's atoms
    conformer = mol.GetConformer()

    # atom-level features and 3D positions
    atom_features, positions = [], []
    for atom in mol.GetAtoms():
        atomic_num = atom.GetAtomicNum()
        atomic_mass, vdw_radius, valence = ATOM_PROPERTIES.get(
            atomic_num, [0.0, 0.0, 0]
        )

        features = [
            atomic_mass,
            vdw_radius,
            valence,
            atom.GetFormalCharge(),
            int(atom.GetIsAromatic()),
            atom.GetDegree(),
        ] + [
            1.0 if atom.GetHybridization() == h else 0.0
            for h in (
                Chem.rdchem.HybridizationType.SP,
                Chem.rdchem.HybridizationType.SP2,
                Chem.rdchem.HybridizationType.SP3,
            )
        ]

        atom_features.append(features)

        pos = conformer.GetAtomPosition(atom.GetIdx())
        positions.append([pos.x, pos.y, pos.z])

    # transform to PyTorch tensors
    x = torch.tensor(atom_features, dtype=torch.float)
    pos = torch.tensor(positions, dtype=torch.float)

    # bonds between atoms – indices of connected atoms as well as types and conjugation
    edge_index, edge_attr = [], []
    for bond in mol.GetBonds():
        # indices of bonded atoms
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()

        # one-hot encode bond type
        bond_type = bond.GetBondType()
        bond_type_one_hot = [1.0 if bond_type == b else 0.0 for b in BOND_TYPES]

        is_conjugated = 1.0 if bond.GetIsConjugated() else 0.0

        bond_feat = bond_type_one_hot + [is_conjugated]

        # adding bond to both nodes
        edge_index += [[i, j], [j, i]]
        edge_attr += [bond_feat, bond_feat]

    # transform to PyTorch tensors
    # edge_index tensor is transposed to fit torch_geometric's expected shape (2, number_of_edges).
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # graph as PyTorch Geometric Data object
    # x: atom features, [atomic number, degree, formal charge, hybridization]
    # pos: 3D positions of atoms, [x, y, z]
    # edge_index: connectivity indices between atoms, [[i, j], [j, i]]
    # edge_attr: features per bond, [[bond type, conjugation], [bond type, conjugation]]
    return Data(
        x=x,
        pos=pos,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y=targets,
        smiles=smiles,
    )


if not torch_data_list:
    torch_data_list = [
        create_torch_data(
            row.canonical_smiles,
            torch.tensor(df_targets.iloc[i].values, dtype=torch.float),
        )
        for i, row in enumerate(df.itertuples(index=False))
    ]

    pickle_file_path = PATH_DATA / "torch_data_list.pkl"

    with open(pickle_file_path, "wb") as f:
        pickle.dump(torch_data_list, f)

    print(f"Saved torch_data_list to {pickle_file_path}")

Create training and test sets.


In [16]:
filtered_torch_data_list = [d.clone() for d in torch_data_list if d is not None]
print(
    f"Number of items in filtered_torch_data_list: {len(filtered_torch_data_list)} / {len(torch_data_list)}"
)  # still retaining original torch_data_list for reference to df later
print(f"filtered_torch_data_list[0]: {filtered_torch_data_list[0]}")

new_filtered_torch_data_list = []

# only include targets with at least n_threshold non-nan values
for d in filtered_torch_data_list:
    d.y = d.y[valid_column_indices]
    if not torch.isnan(d.y).all():
        new_filtered_torch_data_list.append(d)

print(
    f"Number of items in new_filtered_torch_data_list: {len(new_filtered_torch_data_list)} / {len(filtered_torch_data_list)}"
)
filtered_torch_data_list = new_filtered_torch_data_list

split_idx = int(0.8 * len(filtered_torch_data_list))

# data_graph_train = DataLoader(
#     filtered_torch_data_list[:split_idx], batch_size=32, shuffle=True
# )
# data_graph_test = DataLoader(
#     filtered_torch_data_list[split_idx:], batch_size=32, shuffle=False
# )
filtered_torch_data_list_train = filtered_torch_data_list[:split_idx]
filtered_torch_data_list_test = filtered_torch_data_list[split_idx:]

# print(
#     f"# training batches: {len(data_graph_train)}\n# test batches: {len(data_graph_test)}"
# )
print(filtered_torch_data_list_train[0])

Number of items in filtered_torch_data_list: 23439 / 23456
filtered_torch_data_list[0]: Data(x=[49, 9], edge_index=[2, 104], edge_attr=[104, 5], y=[21], pos=[49, 3], smiles='COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC')
Number of items in new_filtered_torch_data_list: 22467 / 23439
Data(x=[49, 9], edge_index=[2, 104], edge_attr=[104, 5], y=[11], pos=[49, 3], smiles='COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC')


## Models


### Setup


In [17]:
if torch.cuda.is_available():
    print("Using CUDA")
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("Using MPS")
    device = torch.device("mps")
else:
    print("Using CPU")
    device = torch.device("cpu")

epochs = 10
n_hidden = 32

n_in = filtered_torch_data_list_train[0].x.shape[1]
n_out = len(valid_column_indices)
n_edge_attr = filtered_torch_data_list_train[0].edge_attr.shape[1]

print(f"Node features: {n_in}, targets: {n_out}, edge attributes: {n_edge_attr}")

Using MPS
Node features: 9, targets: 11, edge attributes: 5


### Naive Baseline


In [31]:
import numpy as np


class MeanBaseline:
    def __init__(self):
        self.mean_ = None

    def fit(self, y):
        self.mean_ = np.nanmean(y, axis=0)

    def predict(self, n):
        return np.tile(self.mean_, (n, 1))


split_idx_baseline = int(0.8 * len(filtered_torch_data_list_train))

y_train_baseline = [
    d.y.numpy() for d in filtered_torch_data_list_train[:split_idx_baseline]
]
y_val_baseline = [
    d.y.numpy() for d in filtered_torch_data_list_train[split_idx_baseline:]
]

naive_baseline = MeanBaseline()
naive_baseline.fit(y_train_baseline)
naive_baseline_predictions = naive_baseline.predict(len(y_val_baseline))

# compute mse
mse_per_target = np.nanmean((y_val_baseline - naive_baseline_predictions) ** 2, axis=0)
print(f"MSE per target: {mse_per_target}")

MSE per target: [1.0670071 1.0954461 2.252836  0.9090124 1.0494274 1.2034993 0.7353269
 2.012234  1.5253873 1.5388899 0.9311896]


### Baseline Model: Random Forest with 2D Descriptors


In [32]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from rdkit.Chem import Descriptors

# tried a bunch of descriptor functions from Descriptors._descList – these are the ones that did NOT crash the kernel ...
safe_descriptors = [
    "MolWt",
    "MolLogP",
    "MolMR",
    "NumValenceElectrons",
    "NumRadicalElectrons",
    "HeavyAtomCount",
    "NHOHCount",
    "NOCount",
    "RingCount",
    "FractionCSP3",
    "TPSA",
    "NumHDonors",
    "NumHAcceptors",
    "NumRotatableBonds",
    "HallKierAlpha",
    "Kappa1",
    "Kappa2",
    "Kappa3",
    "Chi0",
    "Chi1",
    "fr_Al_COO",
    "fr_Al_OH",
    "fr_Ar_N",
    "fr_C_O",
    "fr_NH1",
    "fr_NH2",
]

descriptor_functions = {name: getattr(Descriptors, name) for name in safe_descriptors}


# extract a fixed-length feature vector from the graph data, as input to RF model
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    desc_values = []
    for _, func in descriptor_functions.items():
        try:
            desc_values.append(func(mol))
        except:
            print(f"Error computing descriptor {func}")
    return np.array(desc_values)


X = []
y = []
for data in filtered_torch_data_list_train:
    features = compute_descriptors(data.smiles)
    if features is None:
        continue
    X.append(features)
    target_val = data.y.cpu().numpy() if data.y.numel() > 0 else np.nan
    y.append(target_val)

X = np.array(X)
y = np.array(y)

split_idx_rf = int(0.8 * len(filtered_torch_data_list_train))
X_train, X_val = X[:split_idx_rf], X[split_idx_rf:]
y_train, y_val = y[:split_idx_rf], y[split_idx_rf:]

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print("Val MSE:", mse)

ValueError: Input y contains NaN.

### Approach 1: PyTorch Implementation of a 3D GCN

In this section, a 3D graph convolutional network is created using PyTorch. The model takes as input a 3D molecular graph and outputs predictions of the serotonergic binding affinity of the molecule.

Information about the graph input the model will receive and process:

-   The feature matrix H contains the node (atom) features. Each row corresponds to a node, and each column corresponds to a feature.
-   The adjacency matrix A is built from the edge_index tensor, which contains the indices of the edges in the graph. The matrix A is built under the hood of the GCNConv class.


#### Model Architecture


In [13]:
from torch_geometric.nn import NNConv, GCNConv, global_mean_pool
from torch.nn import Linear
import torch.nn.functional as F


class SeroGCN(torch.nn.Module):
    def __init__(self, n_in, n_hidden, n_out, n_edge_attr):
        super(SeroGCN, self).__init__()

        # edge_network = torch.nn.Sequential(
        #     torch.nn.Linear(n_edge_attr, n_hidden * n_in),
        #     torch.nn.ReLU(),
        # )

        self.pos_lin = Linear(3, n_in)  # simple positional encoding layer

        # self.conv1 = NNConv(n_in, n_hidden, edge_network, aggr="mean")
        self.conv1 = GCNConv(n_in, n_hidden)
        self.conv2 = GCNConv(n_hidden, n_hidden)

        self.fc = Linear(n_hidden, n_out)

    def forward(self, mol_batch) -> torch.Tensor:
        x, pos, edge_index, edge_attr = (
            mol_batch.x,
            mol_batch.pos,
            mol_batch.edge_index,
            mol_batch.edge_attr,
        )

        pos_feat = self.pos_lin(pos)
        x = x + pos_feat  # simple positional encoding

        # x = self.conv1(x, edge_index, edge_attr)
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = global_mean_pool(
            x, mol_batch.batch
        )  # global mean pooling aggregates node features, returning a single graph-level vectorial representation

        x = self.fc(x)

        return x

### Approach 2: Pretraining SeroGCN


#### Downloading the ZINC dataset


In [116]:
from torch_geometric.datasets import ZINC

zinc_dataset = ZINC(root=PATH_DATA / "ZINC", subset=True)
zinc_loader = DataLoader(zinc_dataset, batch_size=32, shuffle=True)

print(zinc_dataset[0])

Data(x=[29, 1], edge_index=[2, 64], edge_attr=[64], y=[1])


In [112]:
n_in_zinc = zinc_dataset.num_node_features
n_edge_attr_zinc = zinc_dataset.num_edge_features
n_out_zinc = zinc_dataset[0].y.shape[0]

print(
    f"Node features: {n_in_zinc}, targets: {n_out_zinc}, edge attributes: {n_edge_attr_zinc}"
)

print(zinc_dataset[0].y)

Node features: 1, targets: 1, edge attributes: 1
tensor([0.8350])


In [100]:
pretrained_model = SeroGCN(
    n_in=n_in_zinc,
    n_hidden=n_hidden,
    n_out=n_out_zinc,
    n_edge_attr=n_edge_attr_zinc,
)
pretrained_model = pretrained_model.to(device)

## Training


In [13]:
def masked_mse_loss(pred, target):
    # mask of non-nan targets
    mask = ~torch.isnan(target)
    if mask.sum() == 0:
        # return 0 loss, so that it doesn't affect the gradient
        return torch.tensor(0.0, requires_grad=True, device=target.device)
    # squared error for entries that are valid
    loss = (pred[mask] - target[mask]) ** 2
    return loss.mean()

In [105]:
def fit(
    model: torch.nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    epochs: int,
):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # --- Training ---
        model.train()
        epoch_loss = 0.0
        start_epoch = time.time()

        for i, data in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            loss = masked_mse_loss(
                out, data.y.view(-1, n_out)
            )  # make sure that even if there's only one target var, it's still a 2D tensor
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

            pct_complete = 100 * (i + 1) / len(train_loader)
            sys.stdout.write(
                f"\rEpoch {epoch+1}/{epochs} - {pct_complete:.2f}% complete"
            )
            sys.stdout.flush()

        train_loss_avg = epoch_loss / len(train_loader)

        # --- Validation ---
        model.eval()
        val_epoch_loss = 0.0
        with torch.no_grad():
            for val_data in val_loader:
                val_data = val_data.to(device)
                val_out = model(val_data)
                val_loss = masked_mse_loss(val_out, val_data.y.view(-1, n_out))
                val_epoch_loss += val_loss.item()
        val_loss_avg = val_epoch_loss / len(val_loader)
        end_epoch = time.time()

        print(
            f"\nEpoch {epoch+1} complete. Train Loss = {train_loss_avg:.4f} | Val Loss = {val_loss_avg:.4f}. Time taken: {end_epoch - start_epoch:.2f}s"
        )
        train_losses.append(train_loss_avg)
        val_losses.append(val_loss_avg)

    return train_losses, val_losses

In [102]:
sero_gcn = SeroGCN(
    n_in=n_in, n_hidden=n_hidden, n_out=n_out, n_edge_attr=n_edge_attr
).to(device)
sero_gcn_optimizer = torch.optim.Adam(sero_gcn.parameters(), lr=0.01)
# sero_gcn_criterion = torch.nn.MSELoss()

In [ ]:
train_val_split = int(0.8 * len(filtered_torch_data_list_train))
train_loader = DataLoader(
    filtered_torch_data_list_train[:train_val_split], batch_size=32, shuffle=True
)
val_loader = DataLoader(
    filtered_torch_data_list_train[train_val_split:], batch_size=32, shuffle=False
)

train_losses, val_losses = fit(
    sero_gcn, train_loader, val_loader, sero_gcn_optimizer, epochs
)

Epoch 1/10 - 100.00% complete
Epoch 1 complete. Train Loss = 1.4948 | Val Loss = 1.2075. Time taken: 105.51s
Epoch 2/10 - 100.00% complete
Epoch 2 complete. Train Loss = 1.3919 | Val Loss = 1.1191. Time taken: 83.97s
Epoch 3/10 - 100.00% complete
Epoch 3 complete. Train Loss = 1.4179 | Val Loss = 1.1003. Time taken: 70.52s
Epoch 4/10 - 100.00% complete
Epoch 4 complete. Train Loss = 1.3945 | Val Loss = 1.2143. Time taken: 91.50s
Epoch 5/10 - 100.00% complete
Epoch 5 complete. Train Loss = 1.3408 | Val Loss = 1.1105. Time taken: 114.29s
Epoch 6/10 - 100.00% complete
Epoch 6 complete. Train Loss = 1.3575 | Val Loss = 1.3576. Time taken: 98.64s
Epoch 7/10 - 100.00% complete
Epoch 7 complete. Train Loss = 1.3483 | Val Loss = 1.0863. Time taken: 88.23s
Epoch 8/10 - 100.00% complete
Epoch 8 complete. Train Loss = 1.3541 | Val Loss = 1.1224. Time taken: 120.38s
Epoch 9/10 - 13.68% complete

KeyboardInterrupt: 